In [2]:
import sys, os
import numpy as np

from time import sleep

## Point to the backend function scripts
sys.path.insert(1, "../DeviceControl")

from NEXUSFunctions import * #control NEXUS fridge

In [3]:
## Temperature scan settings [K]
Temp_base =  11e-3
Temp_min  =  20e-3
Temp_max  = 350e-3
Temp_step =  10e-3

## Temperature stabilization params
tempTolerance =  1e-4     ## K
sleepTime     =  5        ## sec
stabletTime   = 60        ## sec

In [4]:
def temp_change_and_wait(new_sp):

    print("CHANGING SETPOINT TO",new_sp,"K")
    nf.setSP(new_sp)

    cTemp=float(nf.getTemp())
    print("Waiting for Fridge to Reach Temperature")
    print("Monitoring temp every",sleepTime,"seconds")
    print("...",cTemp)
    terr = temp-cTemp
    while(np.abs(terr) > tempTolerance):
        sleep(sleepTime)
        try:
            cTemp=float(nf.getTemp())
            terr = temp-cTemp
            print("...",cTemp,"("+str(terr)+")")
        except:
            print("Socket Failed, skipping reading")

    print("Holding at current temp for",stabletTime,"seconds")
    sleep(stabletTime)

    return 0

In [ ]:
## Initialize the NEXUS temperature servers
nf1 = NEXUSTemps(server_ip="192.168.0.31",server_port=11031)
nf2 = NEXUSTemps(server_ip="192.168.0.32",server_port=11032)

In [ ]:
np.array([nf1.getResistance(), nf2.getResistance()])

In [ ]:
np.array([nf1.getTemp(), nf2.getTemp()])